In [1]:
import czone as cz
import numpy as np
from ase import Atoms
from ase.io import write as ase_write

In [2]:
bbox = cz.volume.makeRectPrism(50,50,50)
Au_gen = cz.generator.from_spacegroup(Z=[79], coords=np.array([[0,0,0]]),\
                                      cellDims=4.07825*np.ones(3), cellAngs=[90,90,90], sgn=225)
pos, species = Au_gen.supply_atoms(bbox)

In [3]:
ase_write("Au_block.xyz", Atoms(symbols=species, positions=pos))

In [4]:
sphere = cz.volume.algebraic.Sphere(center=np.mean(bbox,axis=0), radius=25.0)
Au_sphere = cz.volume.Volume(alg_objects=[sphere], generator=Au_gen, gen_origin=np.array([0,0,0]))
Au_sphere.populate_atoms()

In [5]:
ase_write("Au_sphere.xyz", Atoms(symbols=Au_sphere.species, positions=Au_sphere.atoms))

In [6]:
Cu_gen = cz.generator.from_spacegroup(Z=[29], coords=np.array([[0,0,0]]),\
                                      cellDims=4.07825*np.ones(3), cellAngs=[90,90,90], sgn=225)
small_sphere = cz.volume.algebraic.Sphere(center=np.mean(bbox,axis=0), radius=12.5)
Cu_core = cz.volume.Volume(alg_objects=[small_sphere], generator=Cu_gen, gen_origin=np.array([0,0,0]))

In [7]:
# cutoff_plane = cz.volume.algebraic.Plane(point=sphere.center, normal=np.array([1,0,0]))
# Au_sphere.add_alg_object(cutoff_plane)
# Cu_core.add_alg_object(cutoff_plane)

In [8]:
bounds = np.array([[0.0,0.0,0.0], [30.0, 30.0, 30.0]])
Cu_core.priority = 0
Au_sphere.priority = 1
example_scene = cz.scene.Scene(bounds=bounds, objects=[Au_sphere, Cu_core])

In [9]:
example_scene.populate()

In [10]:
cz.io.write_scene("Au_sphere_Cu_core.xyz", example_scene)

In [11]:
refl_111_plane = cz.volume.algebraic.snap_plane_near_point(sphere.center, Au_gen, (1,1,1))
refl_111_transform = cz.transform.Reflection(refl_111_plane)
vector_112 = np.array([1.0,1.0,-2.0])
vector_121 = np.array([-1.0,2.0,-1.0])
vector_112 *= (4.07825/6)
vector_121 *= (4.07825/6)
translate_112 = cz.transform.Translation(shift=2*vector_112.T)
translate_121 = cz.transform.Translation(shift=2*vector_121.T)

In [12]:
Au_gen_twin = cz.generator.from_generator(Au_gen, transformation=[refl_111_transform])
Au_gen_shift_a = cz.generator.from_generator(Au_gen, transformation=[translate_112])
Au_gen_shift_b = cz.generator.from_generator(Au_gen, transformation=[translate_121])

cutoff_111_a = cz.volume.algebraic.snap_plane_near_point(np.array([32.62, 2.03, 22.43]), Au_gen, (1,1,1))
cutoff_111_b = cz.volume.algebraic.snap_plane_near_point(np.array([20.39, 8.156, 16.313]), Au_gen, (1,1,1))
cutoff_111_b.point += cutoff_111_b.normal

In [13]:
Au_sphere.add_alg_object(refl_111_plane)
Au_sphere_twin = cz.volume.Volume(alg_objects=[sphere], generator=Au_gen_twin, gen_origin=Au_gen_twin.voxel.origin)
Au_sphere_shift_a = cz.volume.Volume(alg_objects=[sphere, cutoff_111_a], generator=Au_gen_shift_a, gen_origin=Au_gen_shift_a.voxel.origin)
Au_sphere_shift_b = cz.volume.Volume(alg_objects=[sphere, cutoff_111_b], generator=Au_gen_shift_b, gen_origin=Au_gen_shift_b.voxel.origin)
vol_list = [Au_sphere, Au_sphere_twin, Au_sphere_shift_a, Au_sphere_shift_b]

In [14]:
Au_sphere.priority = 4
Au_sphere_twin.priority = 5
Au_sphere_shift_a.priority = 3
Au_sphere_shift_b.priority = 2
twin_scene = cz.scene.Scene(bounds=bounds, objects=vol_list)

In [15]:
twin_scene.populate()

In [16]:
cz.io.write_scene("Au_twin.xyz", twin_scene)

In [17]:
refl_111_plane.params

(array([[0.24520321],
        [0.24520321],
        [0.24520321]]),
 array([[24.4695],
        [24.4695],
        [24.4695]]))

In [18]:
translate_up = cz.transform.Translation(shift=np.array([12.5, 12.5, 50-sphere.center[2]]), locked=True) #shift from sides and above future substrate
for vol in vol_list:
       vol.transform(translate_up)

In [19]:
C_substrate_gen = cz.generator.AmorphousGenerator(min_dist=1.2)

In [20]:
substrate_vol = cz.volume.Volume(points=cz.volume.makeRectPrism(70,70,30), generator=C_substrate_gen)

In [21]:
sub_bounds = np.array([[0.0,0.0,0.0], [70.0, 70.0, 85.0]])
# vol_list.append(substrate_vol)
twin_on_substrate_scene = cz.scene.Scene(bounds=sub_bounds, objects=vol_list)

In [22]:
twin_on_substrate_scene.populate()

In [23]:
cz.io.write_scene("Au_twin_on_substrate.xyz", twin_on_substrate_scene)